In [1]:
import numpy as np
from netwin import *
import os
from scipy.integrate import odeint
import matplotlib.pyplot as plt

In [2]:
root_dir = '/home/chaggar/Documents/Network_Inference'
graph_path = os.path.join(root_dir, 'data/brain_networks/scale1.csv')


In [3]:
class NetworkDiffusion(Model):

    def f(self, p, t, theta):
        k = theta
        du = k * (np.matmul(-self.L, p))
        return du

    def solve(self, p, t, theta):
        return odeint(self.f, p, t, args=(theta,))

    def forward(self, u0, t): 
        p = np.exp(u0[:-1])
        theta = u0[-1]
        #
        # print(u0)
        u = self.solve(p, t, theta) 
        return u.T 

In [4]:
m = NetworkDiffusion(network_path=graph_path)

In [5]:
p = np.zeros([83])
p[40] = 40
k = 5

u0 = np.append(np.log(p), k)

t = np.linspace(0,1,100)

p0 = np.ones([83])
k0 = 1

u_0 = np.append(p0, k0)

In [6]:
m.f(p = p, t = t, theta = k)

array([  200.,     0.,     0.,     0.,   200.,   200.,   200.,   200.,
         200.,   200.,   200.,     0.,   200.,   200.,   200.,   200.,
         200.,   200.,   200.,   200.,     0.,   200.,   200.,   200.,
         200.,   200.,   200.,   200.,   200.,   200.,   200.,   200.,
         200.,   200.,   200.,   200.,   200.,   200.,   200.,   200.,
       -8400.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,   200.,   200.,     0.,   200.,   200.,
         200.,   200.,   200.])

In [7]:
data = m.forward(u0=u0, t=t)

In [8]:
problem = InferenceProblem(inference='VB', model=m, data = data, t=t, init_means=u_0)

In [9]:
fi = fit(problem)